In [1]:

from pathlib import Path
import os, sys, yaml, time, glob, re, json
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, Image

# --- Imposta qui la cartella che contiene 'train_pipeline.py' ---
PROJECT_ROOT = Path.cwd()  # es. Path("/home/giuseppe_bonomo/RatioWaveNet2")
if not (PROJECT_ROOT / "train_pipeline.py").exists():
    print("⚠️ 'train_pipeline.py' non trovato nella dir corrente. Provo in quella superiore...")
    if (PROJECT_ROOT.parent / "train_pipeline.py").exists():
        PROJECT_ROOT = PROJECT_ROOT.parent
    else:
        raise FileNotFoundError("Imposta PROJECT_ROOT alla cartella del repo (con train_pipeline.py)")

sys.path.insert(0, str(PROJECT_ROOT))
print("✅ PROJECT_ROOT:", PROJECT_ROOT)
print("🗂️  Configs YAML:", [p.name for p in (PROJECT_ROOT / "configs").glob("*.yaml")])


✅ PROJECT_ROOT: /home/giuseppe_bonomo/RatioWaveNet2
🗂️  Configs YAML: ['eegnet.yaml', 'ctnet.yaml', 'atcnet.yaml', 'ratiowavenet.yaml', 'tsseffnet.yaml', 'tcformer.yaml', 'basenet.yaml', 'mscformer.yaml', 'eegconformer.yaml', 'shallownet.yaml', 'eegtcnet.yaml']


In [2]:

# ===================== PARAMETRI ESPERIMENTO =====================
MODEL_NAME   = "ratiowavenet"        # es. "tcformer", "atcnet", "eegnet", "ratiowavenet", ...
DATASET      = "bcic2a"          # "bcic2a", "bcic2b", "hgd", "reh_mi", "bcic3"
USE_LOSO     = False             # True=LOSO, False=intra-soggetto
INTERAUG     = None              # True / False / None (usa default da YAML)
GPU_ID       = 0                 # -1 per CPU; 0/1/... per GPU
SEED_LIST    = [0,1,2,3,4]       # Elenco seed da lanciare in sequenza
SEED_SLEEP_S = 2                 # Pausa tra seed (secondi)

SUBJECT_MODE = "one"             # "all" | "one" | "list"
ONE_SUBJECT  = 6
SUBJECT_LIST = [1,2,3]

VERBOSE_TRAIN        = True
PLOT_CM_PER_SUBJECT  = True
PLOT_CM_AVERAGE      = True
SAVE_CHECKPOINTS     = False

OVERRIDE_MAX_EPOCHS  = None
# =================================================================


In [3]:

# Costruisce una CONFIG base (senza seed), replicando la logica in run()
from train_pipeline import train_and_test
import pytorch_lightning as pl
from functools import wraps
from pytorch_lightning.trainer.trainer import Trainer as _PLTrainer

CONFIG_DIR = PROJECT_ROOT / "configs"
config_path = CONFIG_DIR / f"{MODEL_NAME}.yaml"
assert config_path.exists(), f"Config non trovato: {config_path}"

with open(config_path) as f:
    base_cfg = yaml.safe_load(f)

cfg = dict(base_cfg)
if USE_LOSO:
    cfg["dataset_name"] = f"{DATASET}_loso"
    cfg["max_epochs"]   = cfg["max_epochs_loso_hgd"] if DATASET == "hgd" else cfg["max_epochs_loso"]
    if "model_kwargs" in cfg and "warmup_epochs_loso" in cfg["model_kwargs"]:
        cfg["model_kwargs"]["warmup_epochs"] = cfg["model_kwargs"]["warmup_epochs_loso"]
else:
    cfg["dataset_name"] = DATASET
    cfg["max_epochs"]   = cfg["max_epochs_2b"] if DATASET == "bcic2b" else cfg["max_epochs"]

cfg["preprocessing"] = cfg["preprocessing"][DATASET]
cfg["preprocessing"]["z_scale"] = cfg.get("z_scale", cfg["preprocessing"].get("z_scale", False))

if INTERAUG is True:
    cfg["preprocessing"]["interaug"] = True
elif INTERAUG is False:
    cfg["preprocessing"]["interaug"] = False
else:
    cfg["preprocessing"]["interaug"] = cfg.get("interaug", cfg["preprocessing"].get("interaug", False))
cfg.pop("interaug", None)

if SUBJECT_MODE == "all":
    cfg["subject_ids"] = "all"
elif SUBJECT_MODE == "one":
    cfg["subject_ids"] = ONE_SUBJECT
elif SUBJECT_MODE == "list":
    cfg["subject_ids"] = SUBJECT_LIST
else:
    raise ValueError("SUBJECT_MODE deve essere 'all' | 'one' | 'list'")

cfg["gpu_id"] = GPU_ID
cfg["plot_cm_per_subject"] = bool(PLOT_CM_PER_SUBJECT)
cfg["plot_cm_average"]     = bool(PLOT_CM_AVERAGE)
if SAVE_CHECKPOINTS:
    cfg["save_checkpoint"] = True

if OVERRIDE_MAX_EPOCHS is not None:
    cfg["max_epochs"] = int(OVERRIDE_MAX_EPOCHS)

os.environ["PL_TRAIN_PROGRESS_BAR"] = "1" if VERBOSE_TRAIN else "0"
if not hasattr(_PLTrainer, "_patched_progress_bar"):
    _orig_init = _PLTrainer.__init__
    @wraps(_orig_init)
    def _wrapped_init(self, *args, **kwargs):
        kwargs.setdefault("enable_progress_bar", (os.environ.get("PL_TRAIN_PROGRESS_BAR","1") == "1"))
        kwargs.setdefault("enable_model_summary", False)
        kwargs.setdefault("log_every_n_steps", 1)
        return _orig_init(self, *args, **kwargs)
    _PLTrainer.__init__ = _wrapped_init
    _PLTrainer._patched_progress_bar = True
    print("🔧 Trainer patch applicata (toggle progress bar).")

print("✅ Config base pronta")
print(yaml.dump(cfg, sort_keys=False))


Tensorflow not install, you could not use those pipelines
🔧 Trainer patch applicata (toggle progress bar).
✅ Config base pronta
dataset_name: bcic2a
subject_ids: 6
max_epochs: 1000
max_epochs_2b: 500
max_epochs_loso: 125
max_epochs_loso_hgd: 77
seed: 0
z_scale: true
preprocessing:
  sfreq: 250
  low_cut: null
  high_cut: null
  start: 0.0
  stop: 0.0
  batch_size: 32
  z_scale: true
  interaug: true
model: RatioWaveNet
model_kwargs:
  F1: 32
  temp_kernel_lengths:
  - 20
  - 32
  - 64
  d_group: 16
  D: 2
  pool_length_1: 8
  pool_length_2: 7
  dropout_conv: 0.4
  use_group_attn: true
  q_heads: 4
  kv_heads: 2
  trans_depth: 5
  trans_dropout: 0.4
  tcn_depth: 2
  kernel_length_tcn: 4
  dropout_tcn: 0.3
  lr: 0.0009
  beta_1: 0.5
  weight_decay: 0.001
  optimizer: adam
  scheduler: true
  warmup_epochs: 20
  warmup_epochs_loso: 3
  use_rdwt: true
  rdwt_levels: 4
  rdwt_level_choices:
  - 2
  - 3
  - 4
  - 5
  - 6
  - 7
  - 8
  - 9
  - 10
  rdwt_base_kernel_len: 16
  rdwt_init_dilatio

In [4]:

# Patch write_summary per creare 'results.txt' compatibile
from utils import metrics as _metrics_mod
import numpy as np
from pathlib import Path

if not hasattr(_metrics_mod, "_write_summary_patched"):
    _orig_write_summary = _metrics_mod.write_summary

    def write_results_txt(result_dir, model_name, dataset_name, subject_ids, param_count,
                          test_accs, test_losses, test_kappas, train_times, test_times, response_times):
        result_dir = Path(result_dir)
        lines = []
        lines.append(f"#Params: {param_count}")
        if isinstance(subject_ids, (list, tuple)):
            for sid, acc in zip(subject_ids, test_accs):
                try:
                    sid_int = int(sid)
                except Exception:
                    sid_int = sid
                lines.append(f"Subject {sid_int} => Test Acc: {float(acc):.4f}")
        if len(test_accs):
            lines.append(f"Average Test Accuracy: {np.mean(test_accs):.4f} ± {np.std(test_accs):.4f}")
        if len(test_kappas):
            lines.append(f"Average Test Kappa: {np.mean(test_kappas):.4f} ± {np.std(test_kappas):.4f}")
        (result_dir / "results.txt").write_text("\n".join(lines))

    def _patched_write_summary(result_dir, model_name, dataset_name, subject_ids, param_count,
                               test_accs, test_losses, test_kappas, train_times, test_times, response_times):
        _orig_write_summary(result_dir, model_name, dataset_name, subject_ids, param_count,
                            test_accs, test_losses, test_kappas, train_times, test_times, response_times)
        write_results_txt(result_dir, model_name, dataset_name, subject_ids, param_count,
                          test_accs, test_losses, test_kappas, train_times, test_times, response_times)

    _metrics_mod.write_summary = _patched_write_summary
    _metrics_mod._write_summary_patched = True
    print("📝 Patch write_summary attivata → verrà creato 'results.txt' in ogni run.")
else:
    print("ℹ️ Patch write_summary già attiva.")


📝 Patch write_summary attivata → verrà creato 'results.txt' in ogni run.


In [ ]:

# Multi-seed loop
from datetime import datetime

results_root = PROJECT_ROOT / "results"
seed_to_dir = {}

print("🚀 Avvio multi-seed:", SEED_LIST)
for sd in SEED_LIST:
    run_cfg = dict(cfg)
    run_cfg["seed"] = int(sd)
    start = time.time()
    print(f"\n==================== Seed {sd} ====================")
    train_and_test(run_cfg)
    pattern = f"{run_cfg.get('model', MODEL_NAME)}_{run_cfg['dataset_name']}_seed-{sd}_*"
    candidates = sorted(results_root.glob(pattern), key=os.path.getmtime)
    if not candidates:
        candidates = sorted(results_root.glob(f"*{run_cfg['dataset_name']}*seed-{sd}*"), key=os.path.getmtime)
    latest = candidates[-1] if candidates else None
    seed_to_dir[sd] = latest
    print(f"✅ Seed {sd} completato. Dir: {latest}")
    time.sleep(SEED_SLEEP_S)

print("📂 seed_to_dir:", seed_to_dir)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


🚀 Avvio multi-seed: [0, 1, 2, 3, 4]

==================== Seed 0 ====================

>>> Training on subject: 6
Setting all random seeds to 0, cuda_available=True


48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Using data from preloaded Raw for 48 events and 1000 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Using data from preloaded Raw for 48 events and 100

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 0:   0%|          | 0/9 [00:00<?, ?it/s] 

...

Epoch 6:  44%|████▍     | 4/9 [00:00<00:00,  5.87it/s, val_loss=1.400, val_acc=0.240, train_loss=1.380, train_acc=0.297]

/home/giuseppe_bonomo/miniconda3/envs/tcformer310/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
48 events found on stim channel stim
Event IDs: [1 2 3 4]
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Using data from preloaded Raw for 48 events and 1000 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Using data from preloaded Raw for 48 events and 100

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 9/9 [00:00<00:00, 16.74it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.2430555522441864
       test_kappa          -0.009259223937988281
        test_loss           1.3927446603775024
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [6]:

# Aggregazione: soggetto × seed + riassunto per seed
import re
import pandas as pd
import numpy as np
from pathlib import Path

def parse_results_txt(path):
    txt = Path(path).read_text()
    subj = {}
    for m in re.finditer(r"Subject\s+(\d+)\s*=>\s*Test Acc:\s*([0-9]+\.?[0-9]*)", txt):
        sid = int(m.group(1)); acc = float(m.group(2))
        subj[sid] = acc
    params = None
    m = re.search(r"#Params:\s*([0-9]+)", txt)
    if m: params = int(m.group(1))
    acc_mean = acc_std = kappa_mean = kappa_std = None
    m = re.search(r"Average Test Accuracy:\s*([0-9]+\.?[0-9]*)\s*±\s*([0-9]+\.?[0-9]*)", txt)
    if m: acc_mean, acc_std = float(m.group(1)), float(m.group(2))
    m = re.search(r"Average Test Kappa:\s*([0-9]+\.?[0-9]*)\s*±\s*([0-9]+\.?[0-9]*)", txt)
    if m: kappa_mean, kappa_std = float(m.group(1)), float(m.group(2))
    return {"params": params, "subj": subj, "acc_mean": acc_mean, "acc_std": acc_std,
            "kappa_mean": kappa_mean, "kappa_std": kappa_std}

seed_parsed = {}
for sd, d in seed_to_dir.items():
    if d is None: continue
    rfile = Path(d) / "results.txt"
    if rfile.exists():
        seed_parsed[sd] = parse_results_txt(rfile)

all_subjects = sorted({sid for x in seed_parsed.values() for sid in x["subj"].keys()})
acc_mat = []
for sid in all_subjects:
    row = {"subject": sid}
    for sd in SEED_LIST:
        row[f"seed-{sd}"] = seed_parsed.get(sd, {}).get("subj", {}).get(sid, np.nan)
    acc_mat.append(row)
df_subject_seed = pd.DataFrame(acc_mat).set_index("subject").sort_index()
df_subject_seed["mean"] = df_subject_seed.mean(axis=1, skipna=True)
df_subject_seed["std"]  = df_subject_seed.std(axis=1, ddof=0, skipna=True)

from IPython.display import display, Markdown
display(Markdown("### 📊 Accuracy per **soggetto × seed** (con media & std)"))
display(df_subject_seed)

rows = []
for sd in SEED_LIST:
    info = seed_parsed.get(sd, {})
    rows.append({
        "seed": sd,
        "#params": info.get("params"),
        "acc_mean": info.get("acc_mean"),
        "acc_std": info.get("acc_std"),
        "kappa_mean": info.get("kappa_mean"),
        "kappa_std": info.get("kappa_std"),
    })
df_seed_summary = pd.DataFrame(rows).set_index("seed")
display(Markdown("### 🧾 Riassunto **per seed**"))
display(df_seed_summary)

export_root = PROJECT_ROOT / "results_aggregates"
export_root.mkdir(parents=True, exist_ok=True)
csv1 = export_root / f"{MODEL_NAME}_{cfg['dataset_name']}_subjects_by_seed.csv"
csv2 = export_root / f"{MODEL_NAME}_{cfg['dataset_name']}_seed_summary.csv"
df_subject_seed.to_csv(csv1, float_format="%.4f")
df_seed_summary.to_csv(csv2, float_format="%.4f")
print("💾 CSV salvati in:", export_root)


KeyError: "None of ['subject'] are in the columns"